
# Case QuantumFinance — Classificador de Chamados (Template)

**Objetivo:** construir e avaliar um classificador de assuntos para chamados da QuantumFinance usando **PLN**, **vetorização (n‑grama + métrica)** e **modelo supervisionado**, alcançando **F1 (weighted) ≥ 0.75**.  
Além do modelo clássico (BoW/TF‑IDF), incluir pelo menos **uma abordagem com Embeddings** (**Word2Vec** e/ou **Sentence Transformers/LLM**).  
Separar **75% treino / 25% teste** com `random_state=42` e, ao final, **consolidar o *pipeline do modelo campeão***.

> Este notebook segue as etapas das aulas 1–6 (pré‑processamento, vetorização, modelos clássicos, embeddings e LLMs) e automatiza a comparação de modelos, selecionando o campeão e exibindo seu pipeline.



## 0. Instalação de dependências (execute apenas uma vez por ambiente)


In [ ]:

# Se estiver em ambiente sem internet, comente as linhas de instalação.
# !pip -q install scikit-learn pandas numpy matplotlib nltk gensim unidecode
# !pip -q install sentence-transformers # (para embeddings LLM)
# !pip -q install spacy
# !python -m spacy download pt_core_news_sm



## 1. Imports e Setup


In [ ]:

import os, re, math, json, random
import numpy as np
import pandas as pd

from unidecode import unidecode
import matplotlib.pyplot as plt

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

# NLTK
import nltk
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import RSLPStemmer

# Gensim (Word2Vec)
from gensim.models import Word2Vec

# Sentence Transformers (LLM Embeddings) - requer internet para baixar no primeiro uso
try:
    from sentence_transformers import SentenceTransformer
    _HAS_ST = True
except Exception as _e:
    _HAS_ST = False

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

# Stopwords PT
STOP_PT = set(stopwords.words('portuguese'))

print("Setup concluído. SentenceTransformer disponível?", _HAS_ST)



## 2. Carregamento dos dados e EDA rápida

> Dataset oficial: `https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv`  
> **Dica:** se o ambiente bloquear internet, baixe o CSV e aponte `DATA_PATH` para o arquivo local.


In [ ]:

# Caminho dos dados — ajuste se necessário
DATA_PATH = "tickets_reclamacoes_classificados.csv"  # ou use a URL direta se o ambiente permitir

# Leitura
try:
    df = pd.read_csv(DATA_PATH)
except Exception as e:
    print("Falha ao ler o CSV no caminho atual. Tente baixar o arquivo localmente ou usar a URL.")
    print("Erro:", e)
    # Tente URL direta
    try:
        df = pd.read_csv("https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv")
        print("Lido via URL com sucesso.")
    except Exception as e2:
        raise RuntimeError("Não foi possível carregar o dataset automaticamente. Baixe o CSV e ajuste DATA_PATH.") from e2

print(df.head())
print(df.info())

# Inferir nomes de colunas esperadas
# Pressupõe colunas como: 'texto' (ou 'mensagem'/'descricao' etc.) e 'assunto' (rótulo)
col_text_candidates = [c for c in df.columns if c.lower() in ("texto","mensagem","descricao","descricao_texto","conteudo")]
col_label_candidates = [c for c in df.columns if c.lower() in ("assunto","categoria","classe","label","rotulo")]

if not col_text_candidates or not col_label_candidates:
    print("\nATENÇÃO: Não identifiquei colunas padrão. Ajuste os nomes abaixo.")
    TEXT_COL = "texto"
    LABEL_COL = "assunto"
else:
    TEXT_COL = col_text_candidates[0]
    LABEL_COL = col_label_candidates[0]

print(f"Coluna de texto: {TEXT_COL} | Coluna de rótulo: {LABEL_COL}")

# Limpeza básica de nulos
before = len(df)
df = df.dropna(subset=[TEXT_COL, LABEL_COL])
after = len(df)
print(f"Removidos {before-after} registros com nulos em {TEXT_COL}/{LABEL_COL}.")

# Distribuição de classes
print("\nDistribuição de classes:")
print(df[LABEL_COL].value_counts())



## 3. Pré-processamento (Aulas 1–2)

- **lowercase, remoção de pontuação**, **remover acentos** (opcional, com `unidecode`),  
- **tokenização**, **remoção de stopwords**, **stem em PT (RSLP)** ou **lematização** via spaCy (opcional).


In [ ]:

STEMMER = RSLPStemmer()

def basic_normalize(text:str) -> str:
    if not isinstance(text, str):
        text = str(text)
    text = text.lower().strip()
    # Remover acentuação (opcional — comente se não quiser)
    text = unidecode(text)
    # Remover caracteres não alfanuméricos básicos (deixe espaço)
    text = re.sub(r"[^a-z0-9áéíóúãõç\s]", " ", text)  # após unidecode, acentos já se foram; mantido para caso desative
    # Colapsar espaços
    text = re.sub(r"\s+", " ", text).strip()
    return text

def tokenize_pt(text:str):
    return word_tokenize(text, language='portuguese')

def preprocess_tokens(tokens, remove_stop=True, do_stem=True):
    out = []
    for w in tokens:
        if remove_stop and w in STOP_PT:
            continue
        if do_stem:
            try:
                w = STEMMER.stem(w)
            except Exception:
                pass
        if w:
            out.append(w)
    return out

def preprocess_text(text:str, remove_stop=True, do_stem=True, join=True):
    text = basic_normalize(text)
    tokens = tokenize_pt(text)
    tokens = preprocess_tokens(tokens, remove_stop=remove_stop, do_stem=do_stem)
    return " ".join(tokens) if join else tokens

# Exemplo rápido
sample = df[TEXT_COL].iloc[0]
print("Original:", sample[:200], "...")
print("Processado:", preprocess_text(sample)[:200], "...")



## 4. Split (75% treino / 25% teste, `random_state=42`)


In [ ]:

X_raw = df[TEXT_COL].astype(str).tolist()
y = df[LABEL_COL].astype(str).tolist()

X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_raw, y, test_size=0.25, random_state=RANDOM_STATE, stratify=y
)

# Pré-processamento aplicado antes de BoW/TF-IDF (opcional; também testaremos sem)
X_train_pp = [preprocess_text(t) for t in X_train_raw]
X_test_pp  = [preprocess_text(t) for t in X_test_raw]

len(X_train_pp), len(X_test_pp)



## 5. Modelos clássicos (BoW / TF‑IDF) — *Baselines & Grid simples*

Testes com:
- **CountVectorizer** e **TfidfVectorizer**
- n‑gramas `(1,1)` e `(1,2)`
- **LogisticRegression**, **LinearSVC**, **MultinomialNB**, **RandomForest**
- Com **texto bruto** e com **texto pré‑processado**


In [ ]:

def eval_pipeline(vectorizer, clf, Xtr, Xte, ytr, yte, name:str):
    pipe = Pipeline([
        ("vec", vectorizer),
        ("clf", clf)
    ])
    pipe.fit(Xtr, ytr)
    ypred = pipe.predict(Xte)
    f1w = f1_score(yte, ypred, average='weighted')
    return {"name": name, "pipeline": pipe, "f1_weighted": f1w}

results = []

vectorizers = [
    ("count_1", CountVectorizer(ngram_range=(1,1))),
    ("count_12", CountVectorizer(ngram_range=(1,2))),
    ("tfidf_1", TfidfVectorizer(ngram_range=(1,1))),
    ("tfidf_12", TfidfVectorizer(ngram_range=(1,2))),
]

classifiers = [
    ("logreg", LogisticRegression(max_iter=2000, n_jobs=None)),
    ("linsvc", LinearSVC()),
    ("mnb", MultinomialNB()),
    ("rf", RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE))
]

datasets = [("raw", X_train_raw, X_test_raw), ("pp", X_train_pp, X_test_pp)]

for ds_name, Xtr, Xte in datasets:
    for v_name, vec in vectorizers:
        for c_name, clf in classifiers:
            name = f"{ds_name}__{v_name}__{c_name}"
            try:
                res = eval_pipeline(vec, clf, Xtr, Xte, y_train, y_test, name)
                results.append(res)
                print(f"{name}: F1w={res['f1_weighted']:.4f}")
            except Exception as e:
                print(f"Falhou {name}: {e}")

df_results_baseline = pd.DataFrame([{"name": r["name"], "f1_weighted": r["f1_weighted"]} for r in results]).sort_values("f1_weighted", ascending=False)
df_results_baseline.head(10)



## 6. Embeddings — **Word2Vec** (média de vetores por documento)

- Treina Skip‑gram (`sg=1`) no corpus pré‑processado.
- Vetor de documento = **média** dos vetores de tokens presentes.
- Classificador: **LogisticRegression** (padrão comparável).


In [ ]:

# Treinar Word2Vec no corpus pré-processado
tokens_train = [preprocess_text(t, join=False) for t in X_train_raw]
tokens_test  = [preprocess_text(t, join=False) for t in X_test_raw]

w2v = Word2Vec(
    sentences=tokens_train,
    vector_size=100,
    window=5,
    min_count=2,
    sg=1,
    workers=4,
    seed=RANDOM_STATE
)

def sent2vec(tokens, model):
    vecs = [model.wv[w] for w in tokens if w in model.wv]
    if not vecs:
        return np.zeros(model.vector_size, dtype=float)
    return np.mean(vecs, axis=0)

Xtr_w2v = np.vstack([sent2vec(tok, w2v) for tok in tokens_train])
Xte_w2v = np.vstack([sent2vec(tok, w2v) for tok in tokens_test])

clf_w2v = LogisticRegression(max_iter=1000)
clf_w2v.fit(Xtr_w2v, y_train)
yp_w2v = clf_w2v.predict(Xte_w2v)
f1_w2v = f1_score(y_test, yp_w2v, average='weighted')
print(f"Word2Vec + LogReg -> F1 (weighted): {f1_w2v:.4f}")



## 7. Embeddings — **Sentence Transformers (LLM)**

Modelo recomendado (multilíngue):  
- `'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'`  
ou `'sentence-transformers/distiluse-base-multilingual-cased-v2'`.

> **Observação:** na primeira execução, o modelo será baixado da internet.


In [ ]:

st_f1 = None
st_model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

if _HAS_ST:
    try:
        st_model = SentenceTransformer(st_model_name)
        Xtr_st = st_model.encode(X_train_raw, show_progress_bar=True)
        Xte_st = st_model.encode(X_test_raw,  show_progress_bar=True)

        clf_st = LogisticRegression(max_iter=2000)
        clf_st.fit(Xtr_st, y_train)
        yp_st = clf_st.predict(Xte_st)
        st_f1 = f1_score(y_test, yp_st, average='weighted')
        print(f"SentenceTransformer + LogReg -> F1 (weighted): {st_f1:.4f}")
    except Exception as e:
        print("Falhou SentenceTransformer:", e)
else:
    print("SentenceTransformer não disponível neste ambiente. Pule esta célula ou instale as dependências.")



## 8. Comparação e seleção do **Modelo Campeão**


In [ ]:

# Agregar resultados
rows = df_results_baseline.copy()
rows["source"] = "baseline"

# Adicionar W2V
rows = pd.concat([
    rows,
    pd.DataFrame([{"name":"w2v__logreg", "f1_weighted": float(f1_w2v), "source":"w2v"}])
], ignore_index=True)

# Adicionar ST se existir
if st_f1 is not None:
    rows = pd.concat([
        rows,
        pd.DataFrame([{"name":"st__logreg", "f1_weighted": float(st_f1), "source":"st"}])
    ], ignore_index=True)

rows = rows.sort_values("f1_weighted", ascending=False).reset_index(drop=True)
rows.head(10)



## 9. **Pipeline do Modelo Campeão** (limpo e pronto para inferência)

Abaixo criamos uma **única versão de pipeline** correspondente ao melhor resultado acima.  
O código identifica se o campeão foi **baseline (TF‑IDF/BoW)**, **Word2Vec** ou **SentenceTransformer**, e então:
- instancia o *transformer* adequado,
- re‑treina no *train*,
- avalia no *test* com relatório,
- expõe função `predict_chamado(textos)` para uso direto.

> Se preferir fixar manualmente o campeão, altere `champion_name`.


In [ ]:

champion_name = rows.iloc[0]["name"]
print("Modelo campeão detectado:", champion_name)

class STVectorizer(BaseEstimator, TransformerMixin):
    """Transformer que converte textos em embeddings usando SentenceTransformer."""
    def __init__(self, model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'):
        self.model_name = model_name
        self.model = None
    def fit(self, X, y=None):
        if self.model is None:
            self.model = SentenceTransformer(self.model_name)
        return self
    def transform(self, X):
        return self.model.encode(list(X), show_progress_bar=False)

class W2VVectorizer(BaseEstimator, TransformerMixin):
    """Transformer que cria embeddings médios Word2Vec."""
    def __init__(self, vector_size=100, window=5, min_count=2, sg=1, seed=RANDOM_STATE):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.sg = sg
        self.seed = seed
        self.model = None
    def fit(self, X, y=None):
        tokens = [preprocess_text(t, join=False) for t in X]
        self.model = Word2Vec(
            sentences=tokens,
            vector_size=self.vector_size,
            window=self.window,
            min_count=self.min_count,
            sg=self.sg,
            workers=4,
            seed=self.seed
        )
        return self
    def transform(self, X):
        tokens_list = [preprocess_text(t, join=False) for t in X]
        def sent2vec(tokens):
            vecs = [self.model.wv[w] for w in tokens if w in self.model.wv]
            if not vecs:
                return np.zeros(self.model.vector_size, dtype=float)
            return np.mean(vecs, axis=0)
        arr = np.vstack([sent2vec(toks) for toks in tokens_list])
        return arr

# Mapeamento para reconstruir pipeline campeão
def build_champion(name:str):
    if name.startswith("st__"):
        if not _HAS_ST:
            raise RuntimeError("SentenceTransformer não disponível para reconstruir o campeão.")
        pipe = Pipeline([
            ("emb", STVectorizer()),
            ("clf", LogisticRegression(max_iter=2000))
        ])
        pipe.fit(X_train_raw, y_train)
        return pipe, "SentenceTransformer + LogReg"
    elif name.startswith("w2v__"):
        pipe = Pipeline([
            ("emb", W2VVectorizer()),
            ("clf", LogisticRegression(max_iter=1000))
        ])
        pipe.fit(X_train_raw, y_train)
        return pipe, "Word2Vec + LogReg"
    else:
        # Baselines: descobrir configuração aproximada pelo sufixo
        # Formato esperado: ds__vec__clf (ex.: "pp__tfidf_12__logreg")
        parts = name.split("__")
        ds, vec_tag, clf_tag = parts[0], parts[1], parts[2]

        # Escolher dataset
        Xtr = X_train_pp if ds == "pp" else X_train_raw

        # Escolher vetorizador
        if vec_tag.startswith("tfidf"):
            ngram = (1,2) if "12" in vec_tag else (1,1)
            vec = TfidfVectorizer(ngram_range=ngram)
        elif vec_tag.startswith("count"):
            ngram = (1,2) if "12" in vec_tag else (1,1)
            vec = CountVectorizer(ngram_range=ngram)
        else:
            raise ValueError("Vec tag desconhecido:", vec_tag)

        # Escolher classificador
        if clf_tag == "logreg":
            clf = LogisticRegression(max_iter=2000)
        elif clf_tag == "linsvc":
            clf = LinearSVC()
        elif clf_tag == "mnb":
            clf = MultinomialNB()
        elif clf_tag == "rf":
            clf = RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE)
        else:
            raise ValueError("Classifier tag desconhecido:", clf_tag)

        pipe = Pipeline([("vec", vec), ("clf", clf)])
        pipe.fit(Xtr, y_train)
        return pipe, f"{vec.__class__.__name__} + {clf.__class__.__name__} ({'PP' if ds=='pp' else 'RAW'})"

champion_pipe, champion_desc = build_champion(champion_name)

# Avaliação no conjunto de teste apropriado ao pipeline
def choose_test_input(pipe, name):
    # Para baselines usamos RAW ou PP conforme detectado no nome
    if name.startswith("st__") or name.startswith("w2v__"):
        return X_test_raw
    else:
        ds = name.split("__")[0]
        return X_test_pp if ds == "pp" else X_test_raw

Xte_input = choose_test_input(champion_pipe, champion_name)
ypred = champion_pipe.predict(Xte_input)
f1w = f1_score(y_test, ypred, average='weighted')
print("\n== CAMPEÃO ==")
print("Descrição:", champion_desc)
print(f"F1 (weighted) no teste: {f1w:.4f}\n")
print(classification_report(y_test, ypred))

cm = confusion_matrix(y_test, ypred, labels=sorted(list(set(y))))
print("Matriz de confusão (ordem de labels alfabética):")
print(cm)

def predict_chamado(textos):
    """Inferência direta no pipeline campeão (lista de strings)."""
    if isinstance(textos, str):
        textos = [textos]
    return champion_pipe.predict(textos)

# Exemplo de inferência:
# predict_chamado(["Não consigo acessar minha conta desde ontem", "Cobrança duplicada no cartão"])



## 10. Conclusões e próximos passos

- **Resumo das técnicas usadas:** pré‑processamento (lower, regex/limpeza, stopwords, stem/lemma), BoW/TF‑IDF, Word2Vec, SentenceTransformer.
- **Comparação de F1 (weighted):** vide tabela na seção 8 para justificar a escolha do campeão.
- **Modelo campeão:** ver seção 9 — pipeline único, limpo e pronto para re‑treino/inferência.
- **Melhorias possíveis:**
  - *Tuning* de hiperparâmetros (C, n_estimators, etc.).
  - Balanceamento de classes (class_weight, amostragem estratificada, técnicas de reamostragem).
  - Lematização com spaCy (PT) + POS‑tag (filtragem por substantivos/verbos).
  - Embeddings mais robustos (ex.: `all-MiniLM-L6-v2` multilíngue; ou BERTimbau no HuggingFace).
  - Validação cruzada e *nested CV* para evitar *overfitting*.
